# Traffic Signs Data Exploration

Let's explore what's in these traffic sign GeoJSON files!

In [ ]:
import json
import pandas as pd
import geopandas as gpd
from pathlib import Path
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

#display options 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Main Categories:
- **A-series**: Speed limits and speed-related signs
- **B-series**: Priority signs (who has right of way)
- **C-series**: Prohibition/restriction signs (no entry, weight limits, etc.)
- **D-series**: Mandatory instructions (roundabouts, directions)
- **E-series**: Parking and stopping regulations
- **F-series**: Other regulatory signs (no overtaking, bus lanes, etc.)
- **G-series**: Road type indicators (motorway, cycle paths, etc.)
- **H-series**: Built-up area signs
- **J-series**: Warning signs (hazards, crossings, etc.)
- **K-series**: Direction and information signs
- **L-series**: Additional/supplementary signs

In [ ]:
#dictionary of common traffic sign meanings
sign_meanings = {
    #A-series: Speed
    'A1': 'Speed limit (e.g., 50 km/h)',
    'A2': 'End of speed limit',
    'A3': 'Speed limit on electronic display',
    'A4': 'Recommended speed',
    'A5': 'End of recommended speed',
    
    #B-series: Priority
    'B1': 'Priority road',
    'B2': 'End of priority road',
    'B3': 'Crossroads with priority',
    'B4': 'Priority over minor road from left',
    'B5': 'Priority over minor road from right',
    'B6': 'Give way sign',
    'B7': 'Stop sign',
    
    #C-series: Prohibitions
    'C1': 'Road closed in both directions',
    'C2': 'No entry (one direction)',
    'C3': 'One-way street',
    'C4': 'One-way street (alternative)',
    'C5': 'Access permitted both sides',
    'C6': 'No vehicles with >2 wheels',
    'C7': 'No goods vehicles',
    'C7a': 'No buses',
    'C7b': 'No buses and goods vehicles',
    'C8': 'No slow vehicles (<25 km/h)',
    'C9': 'No riders, cattle, agricultural vehicles, bikes, mopeds',
    'C10': 'No motor vehicles on wheels',
    'C11': 'No motorcycles',
    'C12': 'No motor vehicles',
    'C13': 'No mopeds',
    'C14': 'No bicycles',
    'C15': 'No bicycles and mopeds',
    'C16': 'No pedestrians',
    'C17': 'No goods vehicles over weight shown',
    'C18': 'No vehicles wider than indicated',
    'C19': 'No vehicles higher than indicated',
    'C20': 'No vehicles over axle load indicated',
    'C21': 'No vehicles over total weight indicated',
    'C22': 'No hazardous substances vehicles',
    'C22a': 'Environmental zone',
    'C22b': 'End of environmental zone',
    
    #D-series: Mandatory
    'D1': 'Roundabout',
    'D2': 'Keep right/left of sign',
    'D3': 'Pass either side',
    'D4': 'Mandatory cycle lane',
    'D5': 'Mandatory path for riders',
    'D6': 'Mandatory path for pedestrians',
    'D7': 'Follow arrow direction',
    
    #E-series: Parking
    'E1': 'No parking',
    'E2': 'No parking and stopping',
    'E3': 'No parking for bicycles/mopeds',
    'E4': 'Parking area',
    'E5': 'Taxi rank',
    'E6': 'Disabled parking',
    'E7': 'Loading/unloading only',
    'E8': 'Parking for specific vehicles',
    'E9': 'Permit holders only',
    'E10': 'Controlled parking zone entry',
    'E11': 'End of controlled parking zone',
    'E12': 'Park and ride',
    'E13': 'Car sharing parking',
    
    #F-series: Other regulations
    'F1': 'No overtaking',
    'F2': 'End of no overtaking',
    'F3': 'No overtaking by trucks',
    'F4': 'End of no overtaking by trucks',
    'F5': 'Give way to oncoming traffic',
    'F6': 'Priority over oncoming traffic',
    'F7': 'No U-turns',
    'F8': 'End of all restrictions',
    'F10': 'Stop (with additional info)',
    'F11': 'Slow vehicle lane',
    'F13': 'Bus lane',
    'F15': 'Tram lane',
    'F19': 'Truck and bus lane',
    'F21': 'Truck lane',
    
    #G-series: Road types
    'G1': 'Motorway',
    'G2': 'End of motorway',
    'G3': 'Expressway',
    'G4': 'End of expressway',
    'G5': 'Living street',
    'G6': 'End of living street',
    'G7': 'Footpath',
    'G8': 'End of footpath',
    'G9': 'Bridleway',
    'G10': 'End of bridleway',
    'G11': 'Cycle route',
    'G12': 'End of cycle route',
    'G12a': 'Cycle and moped route',
    'G12b': 'End of cycle and moped route',
    
    #H-series: Built-up areas
    'H1': 'Built-up area',
    'H2': 'End of built-up area',
    
    #J-series: Warning signs
    'J1': 'Uneven road',
    'J2': 'Bend to right',
    'J3': 'Bend to left',
    'J4': 'Double bend (right first)',
    'J5': 'Double bend (left first)',
    'J6': 'Steep hill upward',
    'J7': 'Steep hill downward',
    'J8': 'Dangerous crossing',
    'J9': 'Roundabout ahead',
    'J10': 'Level crossing with barriers',
    'J11': 'Level crossing without barriers',
    'J12': 'Level crossing single track',
    'J13': 'Level crossing multiple tracks',
    'J14': 'Cattle crossing',
    'J15': 'Falling rocks',
    'J16': 'Slippery road',
    'J17': 'Road narrows both sides',
    'J18': 'Road narrows right',
    'J19': 'Road narrows left',
    'J20': 'Road works',
    'J21': 'Two-way traffic ahead',
    'J22': 'Traffic queues likely',
    'J23': 'Pedestrian crossing',
    'J24': 'Children crossing',
    'J25': 'Cyclist crossing',
    'J26': 'Crossing for riders',
    'J27': 'Wild animals',
    'J28': 'Livestock',
    'J29': 'Two-way traffic',
    'J30': 'Low-flying aircraft',
    'J31': 'Crosswind',
    'J32': 'Traffic lights',
    'J33': 'Drawbridge',
    'J34': 'Danger of accidents',
    'J35': 'Quayside/riverbank',
    'J36': 'Ice or snow risk',
    'J37': 'Danger (general)',
    'J38': 'Speed bump',
    'J39': 'Retractable bollard warning',
    
    #K-series: Direction/information
    'K1': 'Motorway info sign',
    'K2': 'Advance motorway exit sign',
    'K3': 'Service area sign',
    'K4': 'Lane instructions',
    'K5': 'Non-motorway advance info',
    'K6': 'Non-motorway directions',
    'K7': 'Cyclist signposts',
    'K8': 'Multiple cyclist signposts',
    'K9': 'Diversion route',
    'K10': 'Urban area directions',
    'K11': 'Lane instructions (non-motorway)',
    'K12': 'District names',
    'K13': 'District numbers',
    'K14': 'Hazardous materials route',
    
    #L-series: Supplementary
    'L1': 'Height restriction (underpass)',
    'L2': 'Pedestrian crossing',
    'L3': 'Public transport stop',
    'L3a': 'Tram/bus stop',
    'L3b': 'Bus stop',
    'L3c': 'Tram stop',
    'L4': 'Get in lane',
    'L5': 'End of lane',
    'L6': 'Lane fork',
    'L7': 'Number of lanes',
    'L8': 'Pre-sorting',
    'L9': 'No through road',
    'L10': 'Traffic info ahead',
    'L11': 'Lane-specific info',
    'L12': 'Single lane info',
    'L13': 'Tunnel info',
    'L14': 'Hard shoulder',
    'L15': 'Emergency facilities',
    'L16': 'Emergency telephone',
    'L17': 'Fire extinguisher',
    'L18': 'Phone and extinguisher',
    'L19': 'Exit distances',
    'L20': 'Passing area right',
    'L21': 'Passing area left',
    
    'onbekend': 'Unknown/unclassified sign'
}


Sign meaning reference loaded with 166 sign types

Example sign meanings:
  A1: Speed limit (e.g., 50 km/h)
  B6: Give way sign
  C1: Road closed in both directions
  D1: Roundabout
  E4: Parking area
  J23: Pedestrian crossing


In [ ]:
#no more relative paths im done 
DATA_DIR = Path(r'C:\Users\nicol\Documents\TrafficOntology_Project\TrafficOntology\data_processed\traffic_signs_by_type')

files = sorted(DATA_DIR.glob('traffic_signs_*.geojson'))
print(f"Found {len(files)} files\n")

#overview
for i, f in enumerate(files[:10]):  # Show first 10
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"{f.name}: {size_mb:.2f} MB")
    
if len(files) > 10:
    print(f"... and {len(files) - 10} more files")

Found 175 files

traffic_signs_A1.geojson: 98.97 MB
traffic_signs_A2(Zone).geojson: 0.03 MB
traffic_signs_A2.geojson: 45.32 MB
traffic_signs_A3.geojson: 13.61 MB
traffic_signs_A4.geojson: 3.38 MB
traffic_signs_A5.geojson: 0.32 MB
traffic_signs_B1.geojson: 34.30 MB
traffic_signs_B2.geojson: 1.56 MB
traffic_signs_B3.geojson: 8.27 MB
traffic_signs_B4.geojson: 8.16 MB
... and 165 more files


test with A1

In [ ]:
sample_file = DATA_DIR / 'traffic_signs_A1.geojson'
print(f"Loading {sample_file.name}...")

gdf = gpd.read_file(sample_file)
print(f"\nLoaded {len(gdf)} records")
print(f"Columns: {list(gdf.columns)}")

Loading traffic_signs_A1.geojson...

Loaded 153891 records
Columns: ['id', 'externalId', 'validated', 'validatedOn', 'rvvCode', 'blackCode', 'zoneCode', 'status', 'textSigns', 'placement', 'side', 'bearing', 'fraction', 'drivingDirection', 'roadName', 'roadType', 'roadNumber', 'roadSectionId', 'nwbVersion', 'countyName', 'countyCode', 'townName', 'bgtCode', 'imageUrl', 'firstSeenOn', 'lastSeenOn', 'placedOn', 'expectedPlacedOn', 'expectedRemovedOn', 'removedOn', 'trafficOrderUrl', 'geometry']


In [ ]:
display(gdf.drop('geometry', axis=1).head())

,id,externalId,validated,validatedOn,rvvCode,blackCode,zoneCode,status,textSigns,placement,side,bearing,fraction,drivingDirection,roadName,roadType,roadNumber,roadSectionId,nwbVersion,countyName,countyCode,townName,bgtCode,imageUrl,firstSeenOn,lastSeenOn,placedOn,expectedPlacedOn,expectedRemovedOn,removedOn,trafficOrderUrl
0,00005004-4183-409a-aa4a-7dca28977bfa,None,n,NaT,A1,60,ZB,PLACED,[ ],L,N,0.0,0.050776,H,Linnerweg,NaN,NaN,388098016.0,2024-10-01,Roerdalen,GM1669,Montfort,None,https://wegkenmerken.ndw.nu/api/images/e950cfe...,2016-04-19,2024-11-04,NaT,NaT,NaT,NaT,None
1,000069c5-ca65-4a33-8928-5c8171d56907,None,n,NaT,A1,30,ZO,PLACED,[ ],L,Z,180.0,NaN,None,De Ruijterstraat,NaN,NaN,NaN,NaT,Katwijk,GM0537,Rijnsburg,None,https://wegkenmerken.ndw.nu/api/images/ddb6b9b...,2016-04-14,2022-04-13,NaT,NaT,NaT,NaT,None
2,0001301e-2c56-4846-a534-2cf3d96c0ece,None,n,NaT,A1,60,ZB,PLACED,[ ],L,W,270.0,0.706487,T,Pannenhoefsebaan,NaN,NaN,212183001.0,2024-10-01,Zundert,GM0879,Rijsbergen,None,https://wegkenmerken.ndw.nu/api/images/fe84e8a...,2016-08-30,2024-11-04,NaT,NaT,NaT,NaT,None
3,00014dc2-9469-43d7-825e-bc778d8e6d38,None,n,NaT,A1,30,ZB,PLACED,[ ],L,O,90.0,0.477816,H,Eemster,NaN,NaN,442481001.0,2024-10-01,Westerveld,GM1701,Dwingeloo,None,https://wegkenmerken.ndw.nu/api/images/a92862e...,2016-03-31,2024-11-04,NaT,NaT,NaT,NaT,None
4,00016d6b-a14d-4c54-9175-9e3aecea8e9f,None,n,NaT,A1,60,ZB,PLACED,[ ],L,O,90.0,0.641086,H,Bernadettelaan,NaN,NaN,408179045.0,2024-10-01,Horst aan de Maas,GM1507,Tienray,None,https://wegkenmerken.ndw.nu/api/images/00527a6...,2016-12-05,2024-11-04,NaT,NaT,NaT,NaT,None


In [ ]:
#col info
for col in gdf.columns:
    if col != 'geometry':
        dtype = gdf[col].dtype
        null_count = gdf[col].isna().sum()
        null_pct = (null_count / len(gdf)) * 100
        unique = gdf[col].nunique()
        
        print(f"\n{col}:")
        print(f"  Type: {dtype}")
        print(f"  Unique values: {unique}")
        print(f"  Nulls: {null_count} ({null_pct:.1f}%)")
        
        # Show sample values
        if unique <= 10:
            print(f"  Values: {list(gdf[col].dropna().unique())}")
        else:
            print(f"  Sample: {list(gdf[col].dropna().head(3).values)}")

Column Information:

id:
  Type: object
  Unique values: 153891
  Nulls: 0 (0.0%)
  Sample: ['00005004-4183-409a-aa4a-7dca28977bfa', '000069c5-ca65-4a33-8928-5c8171d56907', '0001301e-2c56-4846-a534-2cf3d96c0ece']

externalId:
  Type: object
  Unique values: 1381
  Nulls: 151944 (98.7%)
  Sample: ['BAD7CC62-2F5F-4607-8F0D-BFBB2749886D', '514173DF-FF8B-4787-BE78-03B9A14419FD', '889A8DBC-FC9D-4F30-AC08-9D8FA57A5BA4']

validated:
  Type: object
  Unique values: 2
  Nulls: 0 (0.0%)
  Values: ['n', 'j']

validatedOn:
  Type: datetime64[ms]
  Unique values: 160
  Nulls: 151572 (98.5%)
  Sample: [numpy.datetime64('2025-06-06T00:00:00.000'), numpy.datetime64('2024-09-10T00:00:00.000'), numpy.datetime64('2025-06-06T00:00:00.000')]

rvvCode:
  Type: object
  Unique values: 1
  Nulls: 0 (0.0%)
  Values: ['A1']

blackCode:
  Type: object
  Unique values: 15
  Nulls: 816 (0.5%)
  Sample: ['60', '30', '60']

zoneCode:
  Type: object
  Unique values: 3
  Nulls: 42275 (27.5%)
  Values: ['ZB', 'ZO', 'ZH

check file properties

In [ ]:
file_info = []
all_columns = defaultdict(list)

print("Checking all files...")
for f in files:
    sign_type = f.stem.replace('traffic_signs_', '')
    print(f"  {sign_type}...", end='')
    
    try:
        gdf_sample = gpd.read_file(f, rows=1)
        

        with open(f) as file:
            data = json.load(file)
            record_count = len(data['features'])
        
        columns = list(gdf_sample.columns)
        


        for col in columns:
            if col != 'geometry':
                all_columns[col].append(sign_type)
        
        file_info.append({
            'sign_type': sign_type,
            'records': record_count,
            'columns': len(columns),
            'size_mb': f.stat().st_size / (1024 * 1024)
        })
        print(f"✅({record_count} records)")
        
    except Exception as e:
        print(f"🥀Error: {e}")
        file_info.append({
            'sign_type': sign_type,
            'error': str(e)
        })

df_files = pd.DataFrame(file_info)
print("\nDone!")

Checking all files...
  A1... ✓ (153891 records)
  A2(Zone)... ✓ (49 records)
  A2... ✓ (69964 records)
  A3... ✓ (22527 records)
  A4... ✓ (5337 records)
  A5... ✓ (506 records)
  B1... ✓ (55170 records)
  B2... ✓ (2497 records)
  B3... ✓ (13660 records)
  B4... ✓ (13222 records)
  B5... ✓ (18454 records)
  B6... ✓ (187067 records)
  B7... ✓ (4991 records)
  C1... ✓ (22452 records)
  C10... ✓ (335 records)
  C11... ✓ (289 records)
  C12... ✓ (9472 records)
  C13... ✓ (1942 records)
  C14... ✓ (3794 records)
  C15... ✓ (7386 records)
  C16... ✓ (3774 records)
  C17... ✓ (1447 records)
  C18... ✓ (2052 records)
  C19... ✓ (5677 records)
  C2... ✓ (62055 records)
  C20... ✓ (3427 records)
  C21... ✓ (3023 records)
  C22... ✓ (261 records)
  C22a... ✓ (737 records)
  C22b... ✓ (486 records)
  C22c... ✓ (5 records)
  C22d... ✓ (5 records)
  C23-01... ✓ (3 records)
  C3... ✓ (41117 records)
  C4... ✓ (17692 records)
  C5... ✓ (179 records)
  C6... ✓ (3495 records)
  C7... ✓ (15626 records)


In [ ]:
#summary stats 
print(f"Total files: {len(df_files)}")
print(f"Total records: {df_files['records'].sum():,}")
print(f"Total size: {df_files['size_mb'].sum():.2f} MB")

#add meanings
df_files['meaning'] = df_files['sign_type'].map(sign_meanings)

print(f"\nLargest files by record count:")
display(df_files.nlargest(10, 'records')[['sign_type', 'meaning', 'records', 'size_mb']])

#group by category
df_files['category'] = df_files['sign_type'].str[0]
category_names = {
    'A': 'Speed limits',
    'B': 'Priority',
    'C': 'Prohibitions',
    'D': 'Mandatory',
    'E': 'Parking',
    'F': 'Other regulations',
    'G': 'Road types',
    'H': 'Built-up areas',
    'J': 'Warnings',
    'K': 'Directions',
    'L': 'Supplementary',
    'o': 'Unknown'
}

for cat in sorted(df_files['category'].unique()):
    cat_data = df_files[df_files['category'] == cat]
    cat_name = category_names.get(cat, 'Other')
    print(f"\n{cat}-series ({cat_name}):")
    print(f"  Files: {len(cat_data)}")
    print(f"  Total records: {cat_data['records'].sum():,}")
    print(f"  Signs in this category: {', '.join(sorted(cat_data['sign_type']))}")

FILE SUMMARY WITH SIGN MEANINGS
Total files: 175
Total records: 1,957,371
Total size: 1201.85 MB

Largest files by record count:


,sign_type,meaning,records,size_mb
11,B6,Give way sign,187067,114.059921
0,A1,"Speed limit (e.g., 50 km/h)",153891,98.972697
50,E1,No parking,120378,74.494686
43,D2,Keep right/left of sign,100831,63.943757
95,G11,Cycle route,98533,60.496277
61,E6,Disabled parking,93514,53.364891
174,onbekend,Unknown/unclassified sign,92339,51.393278
2,A2,End of speed limit,69964,45.321862
97,G12a,Cycle and moped route,63845,39.206503
24,C2,No entry (one direction),62055,38.134460



Summary by category:

A-series (Speed limits):
  Files: 6
  Total records: 252,274
  Signs in this category: A1, A2, A2(Zone), A3, A4, A5

B-series (Priority):
  Files: 7
  Total records: 295,061
  Signs in this category: B1, B2, B3, B4, B5, B6, B7

C-series (Prohibitions):
  Files: 29
  Total records: 211,746
  Signs in this category: C1, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19, C2, C20, C21, C22, C22a, C22b, C22c, C22d, C23-01, C3, C4, C5, C6, C7, C7a, C7b, C8, C9

D-series (Mandatory):
  Files: 7
  Total records: 141,573
  Signs in this category: D1, D2, D3, D4, D5, D6, D7

E-series (Parking):
  Files: 22
  Total records: 355,349
  Signs in this category: E1, E1(Zone), E10, E1000, E1001, E11, E12, E13, E2, E3, E4, E5, E6, E7, E8, E8-busvracht, E8-vracht, E8a, E8b, E8c, E9, E9(Zone)

F-series (Other regulations):
  Files: 21
  Total records: 14,392
  Signs in this category: F1, F10, F11, F12, F13, F14, F15, F16, F17, F18, F19, F2, F20, F21, F22, F3, F4, F5, F6, F7, F8

G-se

In [ ]:
#common cols
total_files = len(files)
column_frequency = {col: len(files_list) for col, files_list in all_columns.items()}

print("COLUMN ANALYSIS")
print("="*50)
print(f"Total unique columns: {len(column_frequency)}\n")

#cols in all files
universal_columns = [col for col, count in column_frequency.items() if count == total_files]
print(f"Columns in ALL files ({len(universal_columns)}):")
for col in sorted(universal_columns):
    print(f"  - {col}")

#cols in most files 80>
common_columns = [col for col, count in column_frequency.items() 
                  if count > total_files * 0.8 and col not in universal_columns]
if common_columns:
    print(f"\nColumns in >80% of files ({len(common_columns)}):")
    for col in sorted(common_columns):
        count = column_frequency[col]
        print(f"  - {col} ({count}/{total_files} files)")

#doesnt do shit
rare_columns = [col for col, count in column_frequency.items() if count <= 5]
if rare_columns:
    print(f"\nRare columns (in ≤5 files) ({len(rare_columns)}):")
    for col in sorted(rare_columns)[:10]:  # Show first 10
        files_with_col = all_columns[col]
        print(f"  - {col}: {', '.join(files_with_col)}")

COLUMN ANALYSIS
Total unique columns: 31

Columns in ALL files (14):
  - bearing
  - countyCode
  - countyName
  - firstSeenOn
  - id
  - lastSeenOn
  - placement
  - roadName
  - rvvCode
  - side
  - status
  - textSigns
  - townName
  - validated

Columns in >80% of files (8):
  - drivingDirection (171/175 files)
  - fraction (173/175 files)
  - imageUrl (173/175 files)
  - nwbVersion (173/175 files)
  - placedOn (153/175 files)
  - roadNumber (153/175 files)
  - roadSectionId (174/175 files)
  - roadType (152/175 files)


more on sign types i dont fully get it


In [ ]:
sample_types = ['A1', 'B1', 'C1', 'D1', 'E1']

for sign_type in sample_types:
    file_path = DATA_DIR / f'traffic_signs_{sign_type}.geojson'
    
    if file_path.exists():

        gdf_sample = gpd.read_file(file_path, rows=5)
        
        print(f"Columns: {list(gdf_sample.columns)}")
        print(f"\nFirst record (non-null values only):")
        

        first_record = gdf_sample.iloc[0]
        for col, value in first_record.items():
            if col != 'geometry' and pd.notna(value) and str(value).strip():
                print(f"  {col}: {value}")
    else:
        print(f"\nFile for {sign_type} not found")


Sign Type: A1
Columns: ['id', 'externalId', 'validated', 'validatedOn', 'rvvCode', 'blackCode', 'zoneCode', 'status', 'textSigns', 'placement', 'side', 'bearing', 'fraction', 'drivingDirection', 'roadName', 'roadType', 'roadNumber', 'roadSectionId', 'nwbVersion', 'countyName', 'countyCode', 'townName', 'bgtCode', 'imageUrl', 'firstSeenOn', 'lastSeenOn', 'placedOn', 'expectedPlacedOn', 'expectedRemovedOn', 'removedOn', 'trafficOrderUrl', 'geometry']

First record (non-null values only):
  id: 00005004-4183-409a-aa4a-7dca28977bfa
  validated: n
  rvvCode: A1
  blackCode: 60
  zoneCode: ZB
  status: PLACED
  textSigns: [ ]
  placement: L
  side: N
  bearing: 0
  fraction: 0.050776198506355286
  drivingDirection: H
  roadName: Linnerweg
  roadSectionId: 388098016.0
  nwbVersion: 2024-10-01 00:00:00
  countyName: Roerdalen
  countyCode: GM1669
  townName: Montfort
  imageUrl: https://wegkenmerken.ndw.nu/api/images/e950cfec-c9db-4493-9b7a-4a710c60990a
  firstSeenOn: 2016-04-19 00:00:00
  la

geo data

In [ ]:
for f in files[:5]:
    sign_type = f.stem.replace('traffic_signs_', '')
    gdf_sample = gpd.read_file(f, rows=10)
    
    print(f"\n{sign_type}:")
    print(f"  CRS: {gdf_sample.crs}")
    print(f"  Geometry types: {gdf_sample.geometry.geom_type.unique()}")
 
 
 
    if not gdf_sample.empty and gdf_sample.geometry.iloc[0] is not None:
        first_geom = gdf_sample.geometry.iloc[0]
        if first_geom.geom_type == 'Point':
            print(f"  Sample coordinate: ({first_geom.x:.6f}, {first_geom.y:.6f})")

GEOMETRY ANALYSIS

A1:
  CRS: EPSG:4326
  Geometry types: ['Point']
  Sample coordinate: (5.946671, 51.132845)

A2(Zone):
  CRS: EPSG:4326
  Geometry types: ['Point']
  Sample coordinate: (5.365188, 51.718985)

A2:
  CRS: EPSG:4326
  Geometry types: ['Point']
  Sample coordinate: (6.139505, 53.316612)

A3:
  CRS: EPSG:4326
  Geometry types: ['Point']
  Sample coordinate: (4.483522, 52.125836)

A4:
  CRS: EPSG:4326
  Geometry types: ['Point']
  Sample coordinate: (4.768594, 51.595611)


## 7. Create a summary matrix

In [ ]:
#try find patterns

sign_types = sorted(set(st for files_list in all_columns.values() for st in files_list))
columns = sorted(all_columns.keys())

matrix_data = []
for col in columns[:20]:
    row = {'column': col}
    for st in sign_types[:10]:
        row[st] = '✅' if st in all_columns[col] else '🥀'
    matrix_data.append(row)

df_matrix = pd.DataFrame(matrix_data)
df_matrix.set_index('column', inplace=True)

print("Column presence matrix (sample):")
display(df_matrix)

Column presence matrix (sample):


,A1,A2,A2(Zone),A3,A4,A5,B1,B2,B3,B4
column,,,,,,,,,,
bearing,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
bgtCode,✓,✓,,,,,,,,
blackCode,✓,✓,✓,,✓,✓,,,,
countyCode,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
countyName,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
drivingDirection,✓,✓,✓,✓,✓,✓,✓,✓,✓,✓
expectedPlacedOn,✓,,,,,,,,,
expectedRemovedOn,✓,✓,,✓,✓,,✓,,,
externalId,✓,✓,,✓,✓,✓,✓,✓,✓,✓


Cleaninig time!

In [ ]:

files_to_process = []
files_to_skip = []

for f in files:
    sign_type = f.stem.replace('traffic_signs_', '')
    
    #skip onbekend dont need it if we dont know what type of sign it is
    if sign_type.lower() == 'onbekend':
        files_to_skip.append((f, sign_type, 'Unknown sign type'))
        continue
    
    #avoid scope creep
    if sign_type.startswith('L'):
        files_to_skip.append((f, sign_type, 'L-series (supplementary)'))
        continue
    
    #low record count 
    try:
        with open(f) as file:
            data = json.load(file)
            record_count = len(data['features'])
        
        if record_count < 5000:
            files_to_skip.append((f, sign_type, f'Too few records ({record_count})'))
        else:
            files_to_process.append((f, sign_type, record_count))
    except Exception as e:
        files_to_skip.append((f, sign_type, f'Error reading: {e}'))

print(f"Files to process: {len(files_to_process)}")
print(f"Files to skip: {len(files_to_skip)}")

print("\nFiles to KEEP:")
for f, sign_type, count in sorted(files_to_process, key=lambda x: x[1]):
    meaning = sign_meanings.get(sign_type, 'No description')
    print(f"  {sign_type}: {count:,} records - {meaning}")

print("\nFiles to SKIP:")
for f, sign_type, reason in sorted(files_to_skip, key=lambda x: x[1]):
    print(f"  {sign_type}: {reason}")

FILTERING SIGN TYPES
Files to process: 45
Files to skip: 130

Files to KEEP:
  A1: 153,891 records - Speed limit (e.g., 50 km/h)
  A2: 69,964 records - End of speed limit
  A3: 22,527 records - Speed limit on electronic display
  A4: 5,337 records - Recommended speed
  B1: 55,170 records - Priority road
  B3: 13,660 records - Crossroads with priority
  B4: 13,222 records - Priority over minor road from left
  B5: 18,454 records - Priority over minor road from right
  B6: 187,067 records - Give way sign
  C1: 22,452 records - Road closed in both directions
  C12: 9,472 records - No motor vehicles
  C15: 7,386 records - No bicycles and mopeds
  C19: 5,677 records - No vehicles higher than indicated
  C2: 62,055 records - No entry (one direction)
  C3: 41,117 records - One-way street
  C4: 17,692 records - One-way street (alternative)
  C7: 15,626 records - No goods vehicles
  D1: 21,001 records - Roundabout
  D2: 100,831 records - Keep right/left of sign
  D4: 7,576 records - Mandatory c

In [ ]:
#cols to remove
columns_to_remove = ['bearing', 'validated', 'fraction', 'imageURL', 'placement', 'drivingDirection']

print("Columns to remove:")
for col in columns_to_remove:
    print(f"  - {col}")


if files_to_process:
    sample_file, sample_type, _ = files_to_process[0]
    gdf_sample = gpd.read_file(sample_file, rows=1)
    
    original_cols = list(gdf_sample.columns)
    remaining_cols = [col for col in original_cols if col not in columns_to_remove]
    
    print(f"\nSample from {sample_type}:")
    print(f"Original columns ({len(original_cols)}): {original_cols}")
    print(f"\nRemaining columns ({len(remaining_cols)}): {remaining_cols}")
    print(f"\nRemoved: {[col for col in original_cols if col in columns_to_remove]}")

Columns to remove:
  - bearing
  - validated
  - fraction
  - imageURL
  - placement
  - drivingDirection

Sample from A1:
Original columns (32): ['id', 'externalId', 'validated', 'validatedOn', 'rvvCode', 'blackCode', 'zoneCode', 'status', 'textSigns', 'placement', 'side', 'bearing', 'fraction', 'drivingDirection', 'roadName', 'roadType', 'roadNumber', 'roadSectionId', 'nwbVersion', 'countyName', 'countyCode', 'townName', 'bgtCode', 'imageUrl', 'firstSeenOn', 'lastSeenOn', 'placedOn', 'expectedPlacedOn', 'expectedRemovedOn', 'removedOn', 'trafficOrderUrl', 'geometry']

Remaining columns (27): ['id', 'externalId', 'validatedOn', 'rvvCode', 'blackCode', 'zoneCode', 'status', 'textSigns', 'side', 'roadName', 'roadType', 'roadNumber', 'roadSectionId', 'nwbVersion', 'countyName', 'countyCode', 'townName', 'bgtCode', 'imageUrl', 'firstSeenOn', 'lastSeenOn', 'placedOn', 'expectedPlacedOn', 'expectedRemovedOn', 'removedOn', 'trafficOrderUrl', 'geometry']

Removed: ['validated', 'placement', '

In [17]:
OUTPUT_DIR = Path(r'C:\Users\nicol\Documents\TrafficOntology_Project\TrafficOntology\data_processed\traffic_signs_by_type_cleaned')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Output directory: {OUTPUT_DIR}")
print("="*60)

processing_summary = []

for i, (filepath, sign_type, original_count) in enumerate(files_to_process, 1):
    print(f"Processing {i}/{len(files_to_process)}: {sign_type}...", end='')
    
    try:
        gdf = gpd.read_file(filepath)
        
        cols_to_drop = [col for col in columns_to_remove if col in gdf.columns]
        gdf_clean = gdf.drop(columns=cols_to_drop)
        
        #coords i hope i dont regret this tmrw
        if 'geometry' in gdf_clean.columns and not gdf_clean.geometry.empty:
            #lot log
            gdf_clean['longitude'] = gdf_clean.geometry.x
            gdf_clean['latitude'] = gdf_clean.geometry.y
            
            #wkt
            gdf_clean['geometry_wkt'] = gdf_clean.geometry.astype(str)
            
            gdf_csv = gdf_clean.drop(columns='geometry')
        else:
            gdf_csv = gdf_clean
        
        output_file = OUTPUT_DIR / f'traffic_signs_{sign_type}.csv'
        gdf_csv.to_csv(output_file, index=False)
        



        processing_summary.append({
            'sign_type': sign_type,
            'original_records': original_count,
            'cleaned_records': len(gdf_csv),
            'columns_removed': len(cols_to_drop),
            'final_columns': len(gdf_csv.columns),
            'file_size_mb': output_file.stat().st_size / (1024 * 1024)
        })
        
        print(f"✅({len(gdf_csv)} records, {len(gdf_csv.columns)} columns)")
        
    except Exception as e:
        print(f"⛔Error: {e}")
        processing_summary.append({
            'sign_type': sign_type,
            'error': str(e)
        })

print("\nProcessing complete!")

Output directory: C:\Users\nicol\Documents\TrafficOntology_Project\TrafficOntology\data_processed\traffic_signs_by_type_cleaned
Processing 1/45: A1...✅(153891 records, 29 columns)
Processing 2/45: A2...✅(69964 records, 28 columns)
Processing 3/45: A3...✅(22527 records, 24 columns)
Processing 4/45: A4...✅(5337 records, 26 columns)
Processing 5/45: B1...✅(55170 records, 26 columns)
Processing 6/45: B3...✅(13660 records, 25 columns)
Processing 7/45: B4...✅(13222 records, 24 columns)
Processing 8/45: B5...✅(18454 records, 24 columns)
Processing 9/45: B6...✅(187067 records, 29 columns)
Processing 10/45: C1...✅(22452 records, 28 columns)
Processing 11/45: C12...✅(9472 records, 28 columns)
Processing 12/45: C15...✅(7386 records, 26 columns)
Processing 13/45: C19...✅(5677 records, 26 columns)
Processing 14/45: C2...✅(62055 records, 28 columns)
Processing 15/45: C3...✅(41117 records, 28 columns)
Processing 16/45: C4...✅(17692 records, 27 columns)
Processing 17/45: C7...✅(15626 records, 28 colum

In [18]:
#summary of cleaning 
df_summary = pd.DataFrame(processing_summary)
df_summary_clean = df_summary[~df_summary.get('error', pd.Series()).notna()]


print(f"Files processed: {len(df_summary_clean)}")
print(f"Total records: {df_summary_clean['cleaned_records'].sum():,}")
print(f"Total size: {df_summary_clean['file_size_mb'].sum():.2f} MB")
print(f"Average columns per file: {df_summary_clean['final_columns'].mean():.1f}")

df_summary_clean['meaning'] = df_summary_clean['sign_type'].map(sign_meanings)

print("\nProcessed sign types:")
display(df_summary_clean[['sign_type', 'meaning', 'cleaned_records', 'final_columns', 'file_size_mb']].sort_values('cleaned_records', ascending=False))

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).